In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import warnings
from sklearn.cluster import KMeans
from kneed import KneeLocator
import collections
import auxiliary_functions
import pprint
import json
import random
from itertools import combinations

In [2]:
data = pd.read_csv("./data_house/database.csv", sep = ',') 
column_names = data.columns
n = len(data.columns)
print("Dataset shape:", data.shape)

# Generate a random query   
m = random.randint(1,4)
df = data.sample(n = m, axis = 'columns').sample()
row = []
df_fake_queries = pd.DataFrame(index = range(1), columns = column_names)
df_fake_queries.drop(df_fake_queries.columns[df_fake_queries.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

for j in range(len(df.columns)):
    row.append(''.join((str(df.columns[j]),'=',str(df.iloc[0][j]))))
    df_fake_queries[str(df.columns[j])].iloc[0] = df.iloc[0][j]

print('Query: ', row)
print('Dataframe of query: ', df_fake_queries)

Dataset shape: (10000, 13)
Query:  ['nbedrooms=2', 'windows=15', 'price=46']
Dataframe of query:    nrooms nbedrooms nbath   sm garden_sm floors gargae_sm price year windows  \
0    NaN         2   NaN  NaN       NaN    NaN       NaN    46  NaN      15   

  dist_city doors  
0       NaN   NaN  


In [3]:
query = df_fake_queries.dropna(axis = 1)
query_columns = query.columns
query_values = query.values[0]
print(query)

  nbedrooms price windows
0         2    46      15


In [4]:
# We have to check if the query already exists in our query database
queries =  pd.read_csv("./data_house/queries_to_use.csv", sep = ',', index_col = 0)

In [5]:
resData = queries.merge(df_fake_queries, how = 'inner' ,indicator=False)
print("Common rows between two DataFrames...\n",resData)

Common rows between two DataFrames...
 Empty DataFrame
Columns: [query_id, nrooms, nbedrooms, nbath, sm, garden_sm, floors, gargae_sm, price, year, windows, dist_city, doors]
Index: []


In [6]:
comb, length = auxiliary_functions.combination(query_columns)
comb, length

(['nbedrooms',
  'price',
  'windows',
  ('nbedrooms', 'price'),
  ('nbedrooms', 'windows'),
  ('price', 'windows'),
  ('nbedrooms', 'price', 'windows')],
 7)

In [7]:
dict_query = {}
# We create a dictionary with the possible combinations:
comb, l = auxiliary_functions.combination(query_columns)
for i in range(l):
    dict_query.update({str(comb[i]) : []} )
print(dict_query)   
        

{'nbedrooms': [], 'price': [], 'windows': [], "('nbedrooms', 'price')": [], "('nbedrooms', 'windows')": [], "('price', 'windows')": [], "('nbedrooms', 'price', 'windows')": []}


In [12]:
# We can look for queries that share some of the values
# We create a dictionary to update the repeated values:
length = len(query_columns)

dict_query = auxiliary_functions.matching_queries(length, query_columns, query, dict_query, queries)

Case 3: up to 3 common value
Dictionary:  {'nbedrooms': [8, 27, 42, 46, 50, 55, 80, 104, 122, 130, 145, 147, 151, 161, 165, 176, 188, 202, 275, 280, 329, 347, 352, 365, 373, 389, 412, 436, 441, 452, 473, 474, 485, 498, 555, 576, 582, 589, 598, 639, 648, 657, 659, 665, 666, 674, 699, 703, 705, 711, 714, 720, 741, 742, 751, 753, 760, 761, 766, 775, 777, 778, 792, 819, 834, 847, 851, 886, 887, 898, 929, 955, 962, 968, 974, 985, 1002, 1017, 1021, 1028, 1029, 1061, 1145, 1162, 1183, 1191, 1194, 1203, 1237, 1240, 1249, 1278, 1314, 1357, 1362, 1383, 1386, 1387, 1401, 1432, 1438, 1443, 1455, 1466, 1495, 1517, 1535, 1543, 1547, 1551, 1553, 1569, 1593, 1596, 1599, 1604, 1622, 1654, 1671, 1678, 1709, 1715, 1731, 1733, 1741, 1763, 1768, 1771, 1774, 1781, 1782, 1783, 1804, 1828, 1846, 1877, 1893, 1911, 1949, 1991, 1994, 1995], 'price': [530, 851, 1068, 1253, 1477, 1640, 1934], 'windows': [33, 218, 332, 380, 391, 397, 603, 839, 943, 1033, 1617, 1769, 1850, 1863, 1895], "('nbedrooms', 'price')": [851